In [1]:
!/opt/bin/nvidia-smi
!nvcc --version

Tue Apr 14 18:22:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
  !curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
import os

from fastai.vision import *
from fastai.metrics import error_rate

from PIL import Image, ImageChops, ImageEnhance
from pylab import *

In [2]:
!wget 'https://storage.googleapis.com/ml-weights/exp12-export.pkl'
# !wget 'https://storage.googleapis.com/ml-weights/exp10-export.pkl'
# !wget 'https://storage.googleapis.com/ml-weights/exp9-export.pkl'
# !wget 'https://storage.googleapis.com/ml-weights/exp8-export.pkl'
# !wget 'https://storage.googleapis.com/ml-weights/exp7-export.pkl'

--2020-04-14 18:22:40--  https://storage.googleapis.com/ml-weights/exp12-export.pkl
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 2404:6800:4008:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179065271 (171M) [application/octet-stream]
Saving to: ‘exp12-export.pkl’

exp12-export.pkl    100%[===================>] 170.77M  49.1MB/s    in 3.5s    

2020-04-14 18:22:44 (49.1 MB/s) - ‘exp12-export.pkl’ saved [179065271/179065271]



In [0]:
def convert_to_ela(path, resaved_dir=None, quality=95):
    """Converts image by intentionally resaving an image at a known error rate and then computing the difference
    between the two images.

    Params:
    path: path to the image to be converted using ELA
    quality: quality in which to resave the image to. Defaults to 90%.
    """

    fname = path
    resaved_fname = fname.split('.')[0] + '.resaved.jpg'
    ela_fname = fname.split('.')[0] + '.ela.png'

    img = Image.open(fname).convert('RGB')
    img.save(resaved_fname, 'JPEG', quality=quality)

    img_resaved = Image.open(resaved_fname)
    ela_img = ImageChops.difference(img, img_resaved)

    # Gets the the minimum and maximum pixel values for each band in the image.
    extrema = ela_img.getextrema()

    # Calculate max difference between the pixel values in the image
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff

    ela_img = ImageEnhance.Brightness(ela_img).enhance(scale)

    ela_img.save(ela_fname, 'PNG')

    return ela_fname

In [0]:
exp7_path = Path("/root/.fastai/data/Exp7/")
exp8_path = Path("/root/.fastai/data/Exp8/")
exp9_path = Path("/root/.fastai/data/Exp9/")
exp10_path = Path("/root/.fastai/data/Exp10/")
exp11_path = Path("/root/.fastai/data/Exp11/")
exp12_path = Path("/root/.fastai/data/Exp12/")

In [0]:
learn = load_learner(exp12_path)

In [10]:
path = '/content/data/Images/'
folders = os.listdir(path)

count_fake = 0
count_real = 0
conf_list = []

for folder in folders:
    if os.path.isdir(path + folder):
        new_path = os.path.join(path + folder + '/')
        print(new_path)
        for f in os.listdir(new_path):
            if f != '.ipynb_checkpoints':
                img = open_image(new_path + f)
                pred_class,pred_idx,confidence = learn.predict(img)
            
                if str(pred_class) == 'Tampered':
                    if 'Tampered' == folder:
                        count_fake += 1
                    conf = float(confidence[1])
                elif str(pred_class) == 'Authentic':
                    if 'Authentic' == folder:
                        count_real += 1
                    conf = float(confidence[0])
                conf = conf * 100
                
                conf_list.append(conf)
                print('Predicted: {} ------ Confidence: {:.3f}% ------ Actual: {}'.format(pred_class, conf, folder, f))
            

/content/data/Images/Tampered/
Predicted: Tampered ------ Confidence: 78.212% ------ Actual: Tampered
Predicted: Tampered ------ Confidence: 65.051% ------ Actual: Tampered
Predicted: Tampered ------ Confidence: 88.714% ------ Actual: Tampered
Predicted: Tampered ------ Confidence: 97.880% ------ Actual: Tampered
Predicted: Tampered ------ Confidence: 91.658% ------ Actual: Tampered
Predicted: Tampered ------ Confidence: 83.970% ------ Actual: Tampered
Predicted: Tampered ------ Confidence: 94.969% ------ Actual: Tampered
Predicted: Tampered ------ Confidence: 86.984% ------ Actual: Tampered
Predicted: Tampered ------ Confidence: 69.910% ------ Actual: Tampered
Predicted: Tampered ------ Confidence: 64.797% ------ Actual: Tampered
Predicted: Tampered ------ Confidence: 68.468% ------ Actual: Tampered
Predicted: Tampered ------ Confidence: 88.722% ------ Actual: Tampered
Predicted: Tampered ------ Confidence: 79.758% ------ Actual: Tampered
Predicted: Tampered ------ Confidence: 81.729%

In [11]:
total_fake = len(os.listdir(str(path) + '/Tampered/'))
total_real = len(os.listdir(str(path) + '/Authentic/'))
average_conf = sum(conf_list) / len(conf_list)

accuracy_fake = (count_fake/total_fake) * 100
accuracy_real = (count_real/total_real) * 100

print('Images Classified As Authentic')
print('{}/{}. Accuracy = {:.2f}%'.format(count_real, total_real, accuracy_real))

print('Images Classified As Tampered')
print('{}/{}. Accuracy = {:.2f}%'.format(count_fake, total_fake, accuracy_fake))
print('Average confidence of the model = {:.3f}%'.format(average_conf))

Images Classified As Authentic
16/50. Accuracy = 32.00%
Images Classified As Tampered
44/50. Accuracy = 88.00%
Average confidence of the model = 81.446%


In [0]:
ela_img = convert_to_ela('/content/fake_mercedes.jpg')

In [0]:
img = open_image(ela_img)

In [0]:
len(os.listdir('/content/data/Images/Tampered/'))

100

In [0]:
!rm -rf '/content/data/Images/'

In [0]:
pred_class,pred_idx,conf = learn.predict(img)
pred_class

Category Tampered